In [39]:
import pandas as pd
from collections import defaultdict
import json

In [40]:
# full = pd.read_csv("../data_prep/clean/cleaned.csv", skiprows=range( int(2974335 * .001), 2974335))
full = pd.read_csv("../data_prep/clean/cleaned.csv")

In [41]:
# Question: Which state has the worst drivers?
# Quantized by crashed per capita
# Data Source: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html#par_textimage_1873399417
# Population, Population Change, and Estimated Components of Population Change: April 1, 2010 to July 1, 2019 (NST-EST2019-alldata)
pop = pd.read_csv("../data_prep/other_sources/population_by_state.csv")


state_counts = defaultdict(int)
for k,v in full["State"].value_counts().to_dict().items():
    state_counts[k] = v

state_code = pop["STATECODE"]
# POPESTIMATE MEAN
cols = [f"POPESTIMATE{x}" for x in [2016, 2017, 2018, 2019]] 
pop["4yrpopsum"] = pop[cols].sum(axis=1)
# Over 4 years how many people lived in each state
state_info = pop["4yrpopsum"]
# Contains a list of all the state abbreviations
state_code
# State counts maps state code to number of accidents
state_counts
# Maps all abbreviations to the number of accidents
counts = state_code.apply(lambda code: state_counts.get(code)).fillna(1)
state_info

# Contains chart of accidents per capita
worst_drivers = pd.DataFrame(counts / state_info, columns=["AccidentPC"])
worst_drivers["State"] = state_code

worst_drivers = worst_drivers[worst_drivers.State != "NAN"].sort_values("AccidentPC", ascending=False)

In [42]:
worst_drivers

,AccidentPC,State
45,7.188613e-03,SC
42,4.212232e-03,OR
9,4.209444e-03,CA
38,3.449833e-03,NC
49,3.288324e-03,UT
41,3.255949e-03,OK
32,2.929734e-03,NE
23,2.812376e-03,LA
28,2.808122e-03,MN
14,2.650122e-03,FL


In [3]:
# {
#   "type": "FeatureCollection",
#   "features": [
#     {
#       "geometry": {
#         "type": "Point",
#         "coordinates": [
#           -73.8581124527,
#           40.7177680369
#         ]
#       },
#       "type": "Feature",
#       "id": "1",
#       "properties": {
#         "severity" : "1"
#       }
#     },
#     {
#       "geometry": {
#         "type": "Point",
#         "coordinates": [
#           -73.8582794081,
#           40.671813559
#         ]
#       },
#       "type": "Feature",
#       "id": "2",
#       "properties": {
#         "severity": "4"
#       }
#     }
#   ]
# }



#def crash_state(dataset,state_name):
large_dict = {
    "type": "FeatureCollection",
    "features": []
}


def make_point(lat, long, severity):
    point =  {
          "geometry": {
            "type": "Poi
              nt",
            "coordinates": [
              long,
              lat
            ]
          },
          "type": "Feature",
          "id": "2",
          "properties": {
            "severity": severity
          }
        }
    return point

full.columns
all = full[["Start_Lat", "Start_Lng", "Severity"]].apply(lambda row: make_point(row[0], row[1], row[2]), axis=1)
all.iloc[1]
# Goal: 3 map viewable questions
# Goal: 2 extra data questions

{'geometry': {'type': 'Point', 'coordinates': [-82.831184, 39.92805900000001]},
 'type': 'Feature',
 'id': '2',
 'properties': {'severity': 2.0}}

In [43]:
# def generate_state_data():
#     """
#     INPUT: A fully populated table with all accident data
#     OUTPUT: Fills "state_crash_viz" with 50 json files containing all the points for each state
#     """
all_states = list(full.State.unique())
all = pd.DataFrame(full[["Start_Lat", "Start_Lng", "Severity"]]\
    .apply(lambda row: make_point(row[0], row[1], row[2]), axis=1))
all["State"] = full.State

for state in all["State"].unique():
    print(state)
    st_data = all[all.State == state]
    st_dict = {
        "type": "FeatureCollection",
    }
    feat = []
    for p in list(st_data[0]):
        feat.append(p)
    st_dict["features"] = feat
    
    with open(f'./state_crash_viz/{state}.json', 'w') as fp:
        json.dump(st_dict, fp)
        
    
    

    


OH
WV
CA
FL
GA
SC
NE
IA
IL
MO
WI
IN
MI
NJ
NY
CT
MA
RI
NH
PA
KY
MD
VA
DC
DE
TX
WA
OR
AL
NC
MN
OK
LA
TN
UT
CO
AZ
NV
KS
MS
NM
ME
AR
WY
VT
ID
ND
MT
SD
